<div style="width: 100%; clear: both;">
  <div style="float: left; width: 50%;">
    <img src="https://brandemia.org/sites/default/files/sites/default/files/uoc_nuevo_logo.jpg" align="left" style="width: 80%;">
  </div>
  <div style="float: right; width: 50%; text-align: right;">
    <h3 style="text-align: left; font-weight: bold;">Optimización del sistema de bicicletas compartidas en la ciudad de Valencia.</h3>
    <p style="text-align: left; font-weight: bold; font-size: 100%;">Análisis predictivo, rutas de reparto para el balanceo y gestión eficiente de las estaciones.</p>
    <p style="margin: 0; text-align: right;">Jose Luis Santos Durango</p>
    <hr style="border-top: 1px solid #ccc; margin: 10px 0;">
    <p style="margin: 0; padding-top: 22px; text-align:right;">ETL_historico.ipynb · M2.879 · Trabajo Final de Máster · Área 2</p>
    <p style="margin: 0; text-align:right;">2023-2 · Máster universitario en Ciencia de datos (Data science)</p>
    <p style="margin: 0; text-align:right;">Estudios de Informática, Multimedia y Telecomunicación</p>
  </div>
</div>
<div style="width:100%;">&nbsp;</div>

# Preparación de los datos históricos de Valenbisi

En este notebook vamos a realizar el tratamiento de los datos históricos de Valenbisi. Los datos que tenemos, como ya se ha mencionado en el Notebook 00. ETL_other_data.ipynb, son de 4 años (2020-2023) con una granularidad de minuto (580 millones de registros aprox.) Para lograr este objetivo, vamos a descomprimir los datos de las carpetas que tenemos. Después les daremos el fomato deseado y crearemos un dataframe para poder guardarlo en un fichero con todos los datos formateados y compactados en la misma estructura.


1. [Descompresión de los datos históricos](#1)
        1.1 Extracción de los datos históricos de cada estación y almacenamiento en un fichero txt
2. [Creación de un dataframe con los datos históricos](#2)
3. [Conclusiones](#3)

In [1]:
# import ceil
import os
import tarfile
import time
import pandas as pd
import matplotlib.pyplot as plt

<a id='1'></a>
## 1. Descompresión de los datos históricos

Los datos históricos han sido proporcionados en 4 carpetas comprimidas, cada una con el nombre del año de los datos. En nuestro caso, solo vamos a considerar datos de 2021, como ya se ha comentado. La estructura que siguen estos ficheros es la siguiente:

- Año
    - Mes
        - Día
            - Estaciones (numeradas de 1 a 276) en formato txt. Cada fichero txt tiene la siguiente estructura:
                - Fecha y hora
                - Bicicletas disponibles
                - Parking disponibles

Para descomprimir los datos vamos a crear una función que se ejecutará sobre cada uno de los años de la lista que le pasemos, en nuestro caso 2021. Es importante que para poder ejecutar esta función, se hayan guardado previamente los datos en una carpeta llamada data, en el escritorio local de la persona que ejecuta el sript. Esta función generará como resultado otra carpeta llamada data_descomp que contendrá los datos descomprimidos.

In [2]:
def extract_files_tar_gz(data_years):
    """
    Extract files tar.gz for years in data_years list.

    Parameters:
        data_years (list): A list of ints with the years folders to extract

    Raises:
        OSError: if OS is not supported

    """
    # Get the path for desktop folder
    if os.name == 'nt':  # Windows
        desktop_path = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')
    elif os.name == 'posix':  # macOS y Linux
        desktop_path = os.path.join(os.path.join(os.path.expanduser('~')), 'Desktop')
    else:
        raise OSError("Non supported operative system")
    
    # Start the timer
    start_time = time.time()
    
    for year in data_years:
        # Important to store the data folder in desktop
        folder_name = os.path.join(desktop_path,'data/') + str(year) + '.tar.gz'
        
        # Directorio de destino para la extracción
        destination_directory = os.path.join(desktop_path, 'data_descomp')

        # Abrir el archivo tar.gz
        with tarfile.open(folder_name, "r:gz") as file:
            # Extraer todo el contenido en el directorio de destino
            file.extractall(destination_directory)
        print(f"The folder {folder_name} has been descompressed in {destination_directory}")

    # Calcular el tiempo de ejecución
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Execution time: {execution_time} seconds")

In [3]:
# Execution
years = [2021]

# Descompressed data
extract_files_tar_gz(years)

The folder /Users/jose/Desktop/data/2021.tar.gz has been descompressed in /Users/jose/Desktop/data_descomp
Execution time: 39.50362515449524 seconds


<a id='1.1'></a>
### 1.1 Extracción de los datos históricos de cada estación y almacenamiento en un fichero txt

Una vez los datos han sido descomprimidos en el directorio data_descomp guardado en el escritorio vamos a realizar una transformación sobre los mismos para poder obtener una base de datos estándar sobre la que trabajar. La principal transformación que vamos a realizar será centralizar todos los registros separados en ficheros por estaciones en la jerarquía de carpetas Año/Mes/Día/Estación, y añadir la información de la estación a cada registro, para así obtener un fichero en formato texto con las siguientes columnas separadas por comas: ID de la estación, fecha y hora, bicicletas disponibles, bornetas disponibles.

Para abordar esta tarea usaremos dos funciones. Una función nos ayudará a listar los directorios de todos los ficheros, es decir las rutas donde se localizan los archivos con los datos de las estaciones. La segunda función, para cada directorio de fichero con datos, cogerá esos datos y a cada registro le añadirá la información de la estación, volcando después toda la información en un archivo de texto en el directorio principal llamado datos.txt.

In [4]:
def get_directories(data_years):
    """
    Get directories containing data for specified years.

    Parameters:
        data_years (list): A list of integers representing the years.

    Returns:
        list: A list of directory paths containing data for the specified years.

    Raises:
        OSError: If the operating system is not supported.
    """
    if os.name == 'nt':  # Windows
        data_dir = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop') + '/data_descomp'
    elif os.name == 'posix':  # macOS and Linux
        data_dir = os.path.join(os.path.join(os.path.expanduser('~')), 'Desktop') + '/data_descomp'
    else:
        raise OSError("Operating system not supported")
        
    start_directories = []
    for year in data_years:
        data_dir_year = os.path.join(data_dir, str(year))
        for month in os.listdir(data_dir_year):
            if month != ".DS_Store":
                data_dir_month = os.path.join(data_dir_year, month)
                for day in os.listdir(data_dir_month):
                    if day != ".DS_Store":
                        start_directories.append(os.path.join(data_dir_month, day))
    return start_directories

def data_to_txt(data_years):
    """
    Write data from directories to a text file.

    Parameters:
        data_years (list): A list of integers representing the years.

    """
    start_directories = get_directories(data_years)
    output = '/Users/jose/Desktop/data_descomp/datos.txt'  # Path where the text file will be saved
    
    # Start the timer
    start_time = time.time()

    with open(output, 'w') as txt_file:
        for directory in start_directories:
            for station_id in os.listdir(directory):
                station_path = os.path.join(directory, station_id)
                with open(station_path, 'r', encoding='latin-1') as file:
                    lines = file.readlines()
                    total_lines = len(lines)
                    for i in range(0, total_lines, 60):
                        data = lines[i].strip().split(',')
                        data_station_id = [station_id] + data
                        txt_file.write(','.join(data_station_id) + '\n')
    
    # Calculate the execution time
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Execution time: {execution_time} seconds")


In [5]:
# Execution
years = [2021]
data_to_txt(years)

Execution time: 52.79212808609009 seconds


<a id='2'></a>
## 2. Creación de un dataframe con los datos históricos

In [6]:
start_time = time.time()

# Load the TXT file into a DataFrame
output = '/Users/jose/Desktop/data_descomp/datos.txt'
df = pd.read_csv(output, header=None, names=["station_id", "timestamp", "bikes", "parking"])

# Split the timestamp column into separate date and hour columns
df[['date', 'hour']] = df['timestamp'].str.split(' ', expand=True)

# Convert the 'date' column to dd/mm/yyyy format
df['date'] = pd.to_datetime(df['date']).dt.strftime('%d/%m/%Y')

# Convert the 'hour' column to h:mm format
df['hour'] = pd.to_datetime(df['hour']).dt.strftime('%H:%M')

# Display the first few rows of the resulting DataFrame
print(df.head())

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")


   station_id            timestamp  bikes  parking        date   hour
0         135  2021/03/03 00:00:00     16        4  03/03/2021  00:00
1         135  2021/03/03 01:00:00     16        4  03/03/2021  01:00
2         135  2021/03/03 02:00:00     16        4  03/03/2021  02:00
3         135  2021/03/03 03:00:00     16        4  03/03/2021  03:00
4         135  2021/03/03 04:00:00     16        4  03/03/2021  04:00
Execution time: 35.27678203582764 seconds


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2417213 entries, 0 to 2417212
Data columns (total 6 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   station_id  int64 
 1   timestamp   object
 2   bikes       int64 
 3   parking     int64 
 4   date        object
 5   hour        object
dtypes: int64(3), object(3)
memory usage: 110.7+ MB


<a id='3'></a>
## 3. Conclusiones

Tras realizar varios intentos de optimización del código, al tratarse de una gran cantidad de registros es imposible tratar los datos con un notebook de Python ejecutando con los recursos de la CPU local. Por este motivo se decide escalar con el tutor la incidencia al departamento de Big Data para que nos proporcionen un servidor donde podamos ejecutar Spark y guardar la información en ficheros del sistema distribuido de Hadoop. Mientras tanto, la solución que hemos planteado en este Notebook, será procesar los datos cada 60 líneas, tomando así como referencia el valor de la estación cada hora. La idea era procesar los valores medios para cada hora. también podemos comparar con estos datos y calcular así un nuevo parámetro: la desviación estándar de cada estación cada hora de referencia.